In [37]:
import cv2
import torch
import numpy as np

from ultralytics.nn.tasks import DetectionModel

## Chargement du modèle .best

## Définition des classes et du choix de la classe

In [38]:
classe_choisie = "2x4_Jaune"

In [39]:
classes = [
    "1x2_Blanc",
    "1x2_Bleu",
    "1x2_Jaune",
    "1x2_Marron",
    "1x2_Noir",
    "1x2_Rouge",
    "1x2_Vert clear",
    "1x2_Vert dark",
    "1x4_Blanc",
    "1x4_Jaune",
    "1x4_Noir",
    "1x4_Rouge",
    "1x4_Vert clear",
    "1x4_Vert dark",
    "2x2_Blanc",
    "2x2_Bleu",
    "2x2_Jaune",
    "2x2_Marron",
    "2x2_Rouge",
    "2x2_Vert clear",
    "2x2_Vert dark",
    "2x4_Blanc",
    "2x4_Bleu",
    "2x4_Jaune",
    "2x4_Rouge",
    "2x4_Vert dark"
]

## Fonction de traitement pour la video

In [40]:
# Configuration initiale
VIDEO_SOURCE = "/home/dim/clone_repo/BrickSearch/data/raw/videos/lego_video_test.mp4"
CONFIDENCE_THRESHOLD = 0.7
MODEL_NAME = "y12x_100_640_3000_16_0_065"
MODEL_PATH = f"/home/dim/clone_repo/BrickSearch/outputs/output_models/{MODEL_NAME}/weights/best.pt"
OUTPUT_FILE = "/home/dim/clone_repo/BrickSearch/outputs/video_annotees/result_" + VIDEO_SOURCE.replace("/home/dim/clone_repo/BrickSearch/data/raw/videos/", "")

In [41]:
def initialize_models():
    """
    Initialise les modèles .

    Returns:
        yolo_model entrainé lego: 
    """
# Chargement du modèle sur le GPU 
    checkpoint = torch.load(MODEL_PATH, map_location=torch.device('cuda'))

    model = checkpoint['model']

    return model

In [42]:
def initialize_video_capture_and_writer():
    """
    Initialise la capture vidéo et l'écriture vidéo.

    Returns:
        cap: Objet de capture vidéo.
        video_writer: Objet d'écriture vidéo.
    """    
    cap = cv2.VideoCapture(VIDEO_SOURCE)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # type: ignore
    video_writer = cv2.VideoWriter(OUTPUT_FILE, fourcc, fps, (frame_width, frame_height))


    return cap, video_writer

In [43]:
def prepare_frame(frame):
        # réccupération de la taille de l'image initiale
        height_img_init, width_img_init = frame.shape[:2]

        # Conversion de BGR vers RGB et redimensionnement (ici à 640x640, idem à modele entrainné)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_resized = cv2.resize(image_rgb, (640, 640))

        # Conversion en tensor, normalisation, ajout de la dimension batch,
        # transfert sur GPU et conversion en half precision
        input_tensor = torch.from_numpy(image_resized).permute(2, 0, 1).float() / 255.0
        input_tensor = input_tensor.unsqueeze(0)  # Ajout de la dimension batch
        input_tensor = input_tensor.to(torch.device('cuda')).half()  # Passage en fp16

        return input_tensor, height_img_init, width_img_init

In [44]:
def prediction(model, input_tensor): #

    with torch.no_grad():
        predictions = model(input_tensor) #, stream=True, iou=0.45, max_det=1000, vid_stride=5, batch_size=5, agnostic_nms=True)
    
    # Réorganisation de la sortie :
    # La prédiction initiale a la forme [1, 30, 8400]. On souhaite obtenir un tenseur de forme [8400, 30].
    tensor_predictions = predictions[0]
    preds = tensor_predictions.squeeze(0).transpose(0, 1) # shape : [8400, 30]
    # predictions est un tenseur PyTorch, on le convertit en numpy pour faciliter le traitement avec OpenCV
    preds = preds.cpu().detach().numpy()

    return preds

In [45]:
def write_rectangle_and_text(x1, y1, x2, y2, class_idx, max_class_score, frame, height_img_init, width_img_init):
    # on réajuste à la taille de limage initiale
    # x1, y1 : coordonnées du centre de la boîte
    # x2, y2 : dimensions de la boîte
    x1 = (float(x1) * float(width_img_init) / 640.0)
    y1 = (float(y1) * float(height_img_init) / 640.0)
    x2 = (float(x2) * float(width_img_init) / 640.0)
    y2 = (float(y2) * float(height_img_init) / 640.0)

    # On defini les point d'encadrement de la boîte englobante
        # x1, y1 : coordonnées du centre de la boîte
        # x2, y2 : dimensions de la boîte
    start_point = (int(x1-(x2/2)), int(y1-(y2/2)))
    end_point   = (int(x1+(x2/2)), int(y1+(y2/2)))

    # Dessine le rectangle 
    cv2.rectangle(frame, start_point, end_point, color=(102, 178, 255), thickness=2)
    
    # Prépare le texte à annoter (nom de la classe et score)
    label = f"{classes[class_idx]}: {max_class_score:.2f}"
    
    # Place le texte au-dessus de la boîte
    cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, (102, 178, 255), thickness=1)

In [46]:
def annotationner_frame(model, frame, classe_choisie):

    # préparation de l'image à traiter
    input_tensor, height_img_init, width_img_init = prepare_frame(frame)

    # Inference
    preds = prediction(model, input_tensor)

    # Iterration sur les prédictions
    for pred in preds: 
        # Extraction des coordonnées pour la boîte englobante
        x1, y1, x2, y2 = pred[:4]

        # récupèration du score maximum parmi les classes et son indice
        class_scores = pred[4:]
        max_class_score = np.max(class_scores)
        class_idx = np.argmax(class_scores)
        
        # On vérifie que le score de la classe prédite est supérieur au seuil et uniquement pour la classe cherchée
        if max_class_score > CONFIDENCE_THRESHOLD and classe_choisie == classes[class_idx] :
            
            # On dessine la boîte englobante et le texte
            write_rectangle_and_text(x1, y1, x2, y2, class_idx, max_class_score, frame, height_img_init, width_img_init)


In [47]:
def process_video(classe_choisie):
    """
    Traite la vidéo en détectant, suivant et annotant les objets et les poses.
    """

    # Définition des paramètres de la fenêtre d'affichage
    WINDOW_NAME = 'YOLOv8 Detection'
    WINDOW_WIDTH = 800   # Nouvelle largeur souhaitée
    WINDOW_HEIGHT = 600  # Nouvelle hauteur souhaitée
    WINDOW_POS_X = 100   # Position horizontale sur l'écran
    WINDOW_POS_Y = 100   # Position verticale sur l'écran
        
    cap, video_writer = initialize_video_capture_and_writer()
    
    model = initialize_models()
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break


        # annotation des legos
        annotationner_frame(model, frame, classe_choisie)

        # Création et configuration de la fenêtre
        cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(WINDOW_NAME, WINDOW_WIDTH, WINDOW_HEIGHT)
        cv2.moveWindow(WINDOW_NAME, WINDOW_POS_X, WINDOW_POS_Y)
        cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        


    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

In [48]:
process_video(classe_choisie)